In [1]:
import pickle
import numpy as np
from sklearn import preprocessing
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import  classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit

In [2]:
#試取資料
file = open("../log/snake (1).pickle", "rb")
test = pickle.load(file)
file.close()

print(test['scene_info'][0]['snake_body'])
print(test['command'][0])
if (40,30) in test['scene_info'][0]['snake_body']:
    print('?')

[(40, 30), (40, 20), (40, 10)]
DOWN
?


In [4]:
# feature [command ,蛇長, 蘋果x, 蘋果y, 蛇頭x, 蛇頭y, 上身, 下身, 左身, 右身]
feature = np.array([1,2,3,4,5,6,7,8,9,10])

for i in range(1, 517):
    print(i, end='\r')
    file = open("../log/snake ("+str(i)+").pickle", "rb")
    tmp = pickle.load(file)

    total = len(tmp['scene_info']);
    #刪除吃到倒數第二顆蘋果果前的動作
    last_apple = 0
    f_len = len(tmp['scene_info'][total-1]['snake_body'])
    for k in range(2, total):
        if len(tmp['scene_info'][total-k]['snake_body']) < f_len:
            last_apple = total - k
            break
    #print('last_apple', last_apple)
    
    f = np.array([1,2,3,4,5,6,7,8,9,10])
    for j in range(0, last_apple):
        y = 0
        if tmp['command'][j] == 'DOWN': y = 1
        elif tmp['command'][j] == 'UP': y = 2
        elif tmp['command'][j] == 'RIGHT': y = 3
        elif tmp['command'][j] == 'LEFT': y = 4
        else: y = 5
        
        if y == 5: continue
            
        snake_length = len(tmp['scene_info'][j]['snake_body'])
        apple_x = tmp['scene_info'][j]['food'][0]//10
        apple_y = tmp['scene_info'][j]['food'][1]//10
        head_x = tmp['scene_info'][j]['snake_head'][0]//10
        head_y = tmp['scene_info'][j]['snake_head'][1]//10
        
        ub = 0
        count = 0
        for k in range(head_y-1, 0, -1):
            count += 1
            if (head_x*10, k*10) in tmp['scene_info'][j]['snake_body']:
                ub = count
                break
        
        db = 0
        count = 0
        for k in range(head_y+1, 30, 1):
            count += 1
            if (head_x*10, k*10) in tmp['scene_info'][j]['snake_body']:
                db = count
                break
                
        lb = 0
        count = 0
        for k in range(head_x-1, 0, -1):
            count += 1
            if (k*10, head_y*10) in tmp['scene_info'][j]['snake_body']:
                lb = count
                break
                
        rb = 0
        count = 0
        for k in range(head_x+1, 30, 1):
            count += 1
            if (k*10, head_y*10) in tmp['scene_info'][j]['snake_body']:
                rb = count
                break
                
        #print([y, snake_length, apple_x, apple_y, head_x, head_y, ub, db, lb, rb])
        f = np.vstack((f, np.array([y, snake_length, apple_x, apple_y, head_x, head_y, ub, db, lb, rb])))
    
    #print(feature)
    #print(f)
    feature = np.vstack((feature, f[1:]))
        
print(feature)

[[ 1  2  3 ...  8  9 10]
 [ 1  3  2 ...  0  0  0]
 [ 1  3  2 ...  0  0  0]
 ...
 [ 4 95  2 ...  0  0  1]
 [ 4 95  2 ...  0  0  1]
 [ 4 95  2 ...  0  0  1]]


In [5]:
feature = feature[1:]
print(feature.shape)

(1173209, 10)


In [6]:
#列洗牌
np.random.seed(5)
np.random.shuffle(feature)
np.random.seed(97)
np.random.shuffle(feature)
np.random.seed(44)
np.random.shuffle(feature)
print(feature)

#顯示資料數
print(feature.shape)

x_data = feature[:, 1:]
print(x_data)
print(x_data.shape)
y_data = feature[:, 0]
print(y_data)
print(y_data.shape)

[[ 1 65 26 ...  0  0  6]
 [ 4 16  5 ...  0  0  1]
 [ 3 15 19 ...  1  0  0]
 ...
 [ 2 90  2 ...  1  1  1]
 [ 3  9 25 ...  1  0  0]
 [ 1 38 17 ...  0  0  4]]
(1173209, 10)
[[65 26 20 ...  0  0  6]
 [16  5 21 ...  0  0  1]
 [15 19  0 ...  1  0  0]
 ...
 [90  2 20 ...  1  1  1]
 [ 9 25  7 ...  1  0  0]
 [38 17 27 ...  0  0  4]]
(1173209, 9)
[1 4 3 ... 2 3 1]
(1173209,)


In [7]:
t0 = np.sum(y_data == 1)
t1 = np.sum(y_data == 2)
t2 = np.sum(y_data == 3)
t3 = np.sum(y_data == 4)
print(t0, t1, t2, t3)

286884 280402 305785 300138


In [8]:
#儲存陣列
file = open('snake_20200827_perpared_x_data_9_feature.pkl', 'wb')
pickle.dump(x_data, file)
file.close()
file = open('snake_20200827_perpared_y_data_9_feature.pkl', 'wb')
pickle.dump(y_data, file)
file.close()

In [ ]:
#資料劃分
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=9)
#參數區間
param_grid = {'C': [0.1, 1], 'gamma':[0.001, 0.1], 'kernel': ['rbf']}
#交叉驗證 n_splits多可以驗證資料的一致性，但浪費時間效益不大
cv = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=12)
grid = GridSearchCV(svm.SVC(), param_grid, cv=cv, verbose=10, n_jobs=-1)
grid.fit(x_train, y_train)
grid_predictions = grid.predict(x_test)

#儲存
file = open('snake_20200827_data_9_feature_svm_model.pkl', 'wb')
pickle.dump(grid, file)
file.close()